# Opus Method 1

This method of Opus verifying information about an indvidual relies on passing the entire payload received by the SSL component of the https handling mechanism to an Opus server. Opus can then parse the relevant page containing information pertaining to the online identity of an individual. Opus knows this is the same information as was presented by the orginal third party because the key used to encrypt the payload matches the key contained on the SSL certificate owned by that third party. With this trust in place,Opus has everything it needs to issue a credential.

In order to issue a credential, first we need to define it on the ledger:

<img src="../../images/opus_1A.png" alt="drawing" width="500"/>

Next we need to implement a method for moving <b>trusted</b> data from third parties online into our private wallets. Method 1 can be seen below. This strong trust guarentees as Opus is able to verify that the html page presented by the user is in fact the one presented by the third party. However, this also requires that the user sends <i>everything</i> which has been communicated by the thirdparty; including sensitive information like session keys. This method is only applicable where the user trusts Opus with the access to this information.

<img src="../../images/opus_1B.png" alt="drawing" width="800"/>

## 1. Retrieving an Authenticated Github Account Page

This script uses the python Requests library to retrieve a webpage and manage a session. The issue with this is that all TLS is run on wrapped libraries and is not accessible from the Requests object. However, if we can decrypt an https response, we know the cryptography is valid. We could work with PyOpenSSL instead but this seems to be deprecated. Solutions Welcome.

In [ ]:
import requests
from getpass import getpass
from bs4 import BeautifulSoup


headers = {
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'
}

login_data = {
'commit': 'Sign in',
'utf8': '%E2%9C%93',
'login': input('Username: '),
'password': getpass()
}
url = 'https://github.com/session'
session = requests.Session()
response = session.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html5lib')
login_data['authenticity_token'] = soup.find(
'input', attrs={'name': 'authenticity_token'})['value']
response = session.post(url, data=login_data, headers=headers)
print(response.status_code)
response = session.get('https://github.com/settings/organizations', headers=headers)
# print(response.text)

## 2. Pass Complete Response from Third Party to Opus Server

This could be achieved through capturing the still https payload- pre TLS unpacking- and passing relevant components using the basic messaging protocol. This is non-trivial but achievable. For now lets assume this works and work out the logic for how things will happen next.  

## 3. Parse Received Page for Organisation Membership

Here we parse the recieved organisation page (as Opus) and recieve the membership affiliations.

In [10]:
response.text

# from IPython.core.display import display, HTML
# display(HTML(response.text))

'\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-jFUBCdWOA1Ov3xo3oFMBwsdP4Up2K1bRnP4QYI5WqvpaIYxWVek89k2M0oyTbNhYMViGtxJB3Vdwcw8ln8hGQw==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-8c550109d58e0353afdf1a37a05301c2.css" />\n  \n    <link crossorigin="anonymous" media="all" integrity="sha512-UDsGRotwx2k6kbfDX9jHfPF0usc3W8ZycpLWUkGPOl75fCsW4/G/yrOzGIXhNM

In [3]:
soup = BeautifulSoup(response.text)

orgsSection = soup.findAll("div", {"class": "col-9 float-left"})[0].findAll("div", {"class": "Box"})[0].findAll('img')
myOrgs = set(tag['alt'] for tag in orgsSection)
print(myOrgs)


{'@diffusioncon', '@NeverMindTheBlockchain', '@blockpass-identity-lab', '@OpenMined'}


## 4. Return Parsed Data as Signed Credential